Simple notebook for easily viewing different samples of the nuscenes datasets to compile a list of sample tokens of samples to analyse for the evaluation of safety-oriented metrics

In [1]:
# LOAD NUSCENES
from nuscenes import NuScenes
DATAROOT = '/cluster/work/andronn/MasterThesis/MASTER/mmdetection3d/data/nuscenes'
nusc = NuScenes('v1.0-trainval', dataroot=DATAROOT)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 49.540 seconds.
Reverse indexing ...
Done reverse indexing in 17.0 seconds.


Load list of all samples and then randomly shuffle and display samples by running next cell

In [2]:
import random
import os
import json
from json import JSONDecodeError
from IPython.display import clear_output
results_root ='/cluster/work/andronn/MasterThesis/MASTER/master_repo/Thesis-Evaluating-Safety-Oriented-Metrics-for-Object-Detectors/results/'
detectors = ['pointpillars-fpn/', 'ssn-REGNET/', 'regnet-regnetX_FPN/']
data_loc = [results_root+detector+'METRIC_SAMPLES/' for detector in detectors]

# Assumes a saved json with all sample tokens in data_root on form { 'sample_tokens': list(...) }
with open(os.path.join(results_root, 'sample_tokens.json'), 'r') as f:
    sample_tokens = json.load(f)
    
#random.seed(43)
sample_tokens = list(sample_tokens['sample_tokens'])


In [3]:
num_samples = int(input("Input number of samples to view: "))
assert num_samples>0

random.shuffle(sample_tokens)
samples_root ='/cluster/work/andronn/MasterThesis/MASTER/master_repo/Thesis-Evaluating-Safety-Oriented-Metrics-for-Object-Detectors/results/'
fn = os.path.join(samples_root, 'sample_tokens_selected.json')
try:
    pre_saved_samples = None
    with open(fn, 'r') as f:
        pre_saved_samples = json.load(f)
except:
    print("No sample_tokens_selected.json file: proceeding to create one after sample selection.\n")

save_dict =  {'sample_tokens': list()}
if pre_saved_samples:
    for token in pre_saved_samples['sample_tokens']:
        save_dict['sample_tokens'].append(token)


for token in sample_tokens[:num_samples]:
    nusc.render_sample(token)
    for l in range(len(data_loc)):
        with open(os.path.join(results_root, detectors[l], 'pkl_results.json'), 'r') as f:
            print("Detector: {}, PKL: {}".format(detectors[l], json.load(f)['full'][token])) # print sample PKL
            
    x = input("Save current sample for later analyzing?[y/n]")
    clear_output(wait=False)
    if x=="y" or x=="Y":
        save_dict['sample_tokens'].append(token)
        

In [4]:
# Save list of sample tokens to utilize in analysis

with open(fn, 'w+') as f:
    json.dump(save_dict, f, indent=2)